### Introduction to Tools in LangChain

Large Language Models (LLMs) are powerful, but they can't access real-time data or perform actions on their own. **Tools** bridge this gap.

Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or calculating values.

Key concepts:
1.  **Schema**: Defines the tool's name, description, and parameters so the model knows how to use it.
2.  **Function**: The actual Python code that performs the task.
3.  **Tool**: A wrapper around the function that provides the schema and metadata.

### 1. Defining a Tool

In [6]:
import os
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage
from dotenv import load_dotenv

load_dotenv()

# Define the tool with hardcoded logic
@tool
def get_weather(location: str):
    """Get the weather for a specific location."""
    location = location.lower()
    if "london" in location:
        return "It is rainy in London."
    elif "new york" in location:
        return "It is sunny in New York."
    elif "kolkata" in location:
        return "It is sunny in Kolkata."
    else:
        return f"The weather in {location} is partly cloudy."

### 2. Binding Tools to the Model
We need to tell the model about the tools available to it. We do this using `.bind_tools()`. This converts the tool definitions into a format the model understands (like OpenAI function calling schema).

In [5]:
# Initialize model
model = init_chat_model("llama-3.1-8b-instant", model_provider="groq")

# Bind tool to model
model_with_tools = model.bind_tools([get_weather])

model_with_tools

### 3. The Agent Loop (Manual Implementation)
When an agent runs, it typically follows this loop:
1.  **Thinking**: The model receives the user query and decides if it needs to call a tool.
2.  **Tool Call Generation**: If a tool is needed, the model returns a "tool call" request (name of tool + arguments).
3.  **Execution**: We (or the agent runtime) execute the tool with the provided arguments.
4.  **Observation**: The output of the tool is fed back to the model.
5.  **Final Response**: The model uses the tool output to generate a final natural language response for the user.

Below is a manual implementation of this loop:

In [2]:
print("--- Step 1: Model generates tool call ---")
# step1: Model generates tool call
message = [HumanMessage(content="What's the weather in Kolkata?")]
ai_msg = model_with_tools.invoke(message)
message.append(ai_msg)
print(f"AI Message Content: {ai_msg.content}")
if ai_msg.tool_calls:
    print(f"Tool Calls: {ai_msg.tool_calls}")

# step2: Iterate and execute and collect the results
if ai_msg.tool_calls:
    print("\n--- Step 2: Iterate and execute ---")
    for tool_call in ai_msg.tool_calls:
        # execute the tool with generated arguments
        # invoking tool_call directly works if it's a ToolCall dict? No, usually expects args.
        # But let's use the explicit args to be safe and clear.
        tool_output = get_weather.invoke(tool_call)
        print(f"Tool Output: {tool_output}")
        
        # Create a ToolMessage to strictly follow LangChain's message history format
        tool_message = ToolMessage(
            tool_call_id=tool_call["id"],
            content=str(tool_output),
            name=tool_call["name"]
        )
        message.append(tool_message)

# step3: Pass the result back to the model for final response
print("\n--- Step 3: Final Response ---")
final_response = model_with_tools.invoke(message)
# Use .content instead of .text
print(final_response.content)